In [1]:
# TODO: Remove this before submission
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [2]:
import pandas as pd
import numpy as np
import nltk

nltk.download("wordnet")
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

## Read Data


## Keep Reviews and Ratings


In [5]:
# Load the tab separated data file, and print the first 5 rows for confirmation
data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])
data.head()


/var/folders/d7/gcxbjppx70qfkdwy32kg04dh0000gn/T/ipykernel_28649/2961754961.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


Understanding Data


In [6]:
data.describe()


,star_rating,review_body
count,1767042,1766807
unique,11,1618522
top,5,Love it
freq,1041056,4288


In [7]:
data.star_rating.unique()


array([5, 1, 4, 3, 2, nan, '5', '1', '3', '4', '2', '2012-12-21'],
      dtype=object)

In [8]:
data.groupby(["star_rating"]).count()


,review_body
star_rating,
1,150441
2,97259
3,153660
4,259019
5,1040896
1,4566
2,3541
2012-12-21,0
3,5999


In [9]:
# Drop the outlier which is star_rating = "2012-12-21"
data = data[data.star_rating != "2012-12-21"]


In [10]:
# Remove nan valued rows
data = data[data.star_rating.notnull()]


In [11]:
data.describe()


,star_rating,review_body
count,1767041,1766807
unique,10,1618522
top,5,Love it
freq,1041056,4288


In [12]:
# Convert all star rating to integer
data["star_rating"] = data.star_rating.astype(int)


In [13]:
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [14]:
data = data[data.review_body.notnull()]
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [15]:
# There are no empty reviews
(data.review_body.str.len() <= 0).sum()


0

Now we can continue with the process.


## We select 20000 reviews randomly from each rating class.


In [64]:
# np.random.seed(101)
N_SAMPLES = 25000


In [65]:
sampled_data = data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(N_SAMPLES))


In [66]:
# TODO: Remove Test Block
# Checkout: Sampled Review, Ratings Data
# Save data to a file for having a look at the text content

sampled_data.head()
sampled_data.to_csv("./data/sampled.tsv", sep="\t", index=False, encoding="UTF-8")


,star_rating,review_body
748397,1,"much smaller than regular Pandora, read the fi..."
1019734,1,Too small. No satisfied.. need to b bigger wan...
1577357,1,"I ordered this and it was NOT rainbow, even th..."
196612,1,Terrible broke in 3 days
887893,1,This is not a locket. It is completely impossi...


# Data Cleaning


In [67]:
avg_len_before_cleaning = sampled_data.review_body.str.len().mean()
f"Avg. length of reviews BEFORE CLEANING :: {avg_len_before_cleaning}"


'Avg. length of reviews BEFORE CLEANING :: 189.356672'

In [68]:
# TODO: Remove Test Block
# Test to lower case
sampled_data["review_body"].str.lower()


748397     much smaller than regular pandora, read the fi...
1019734    too small. no satisfied.. need to b bigger wan...
1577357    i ordered this and it was not rainbow, even th...
196612                              terrible broke in 3 days
887893     this is not a locket. it is completely impossi...
                                 ...                        
482005              perfect fashion earrings!...great buy!..
1034042    i ordered several sterling silver mens bracele...
162964                                         great quality
1369264    well here i go again with the daughter enjoyin...
1497268    i chose this diamond white gold necklace becau...
Name: review_body, Length: 125000, dtype: object

In [69]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: pd.Series):
    return data.str.lower()


In [70]:
# TODO: Remove Test Block
import unicodedata

# Remove accented characters
print(
    unicodedata.normalize(
        "NFKD", "Sómě Áccěntěd těxt. Some words such as résumé, café, prótest, divorcé, coördinate, exposé, latté."
    )
    .encode("ascii", "ignore")
    .decode("utf-8", "ignore")
)


Some Accented text. Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [71]:
def remove_accented_characters(data: pd.Series):
    """Removes accented characters from the Series

    Args:
        data (pd.Series): Series of string

    Returns:
        _type_: pd.Series
    """
    import unicodedata

    return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))


In [72]:
# TODO: Remove Test Block
# Test HTML and URL replace RegEx
import re

# Removing HTML tags with and without closing tags
re.sub(
    r"<[a-zA-Z]+\s?/?>",
    " ",
    "this is a string <containing> some random XML tags <br> which might not be closed like this <br /> or this <br/>",
)

# Removing HTML encoded characters.
re.sub(r"&#\d+;", " ", "why are &#34; you doing this?")

# Removing URLs
re.sub(
    r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+",
    " ",
    "Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\"
    "The more I give to you, the more I have;<br />For both are infinite.\\"
    "<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Garnet by night:<br />To stand right in Our Saviour's sight;<br />This is our health;<br />And all of our wealth. --- Gwen Newclear The Poet<br /><br />I thought I lost this ring:  found it.  http://poetgwennewclear - dot - sharepoint - dot - com https://www.gogole.com/asd___asd?test=1",
)


'this is a string   some random XML tags   which might not be closed like this   or this  '

'why are   you doing this?'

"Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\The more I give to you, the more I have;<br />For both are infinite.\\<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Ga

In [73]:
def remove_html_and_url(data: pd.Series):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (pd.Series): A Pandas series of type string

    Returns:
        _type_: pd.Series
    """
    # Remove HTML encodings
    data.str.replace(r"&#\d+;", " ", regex=True)

    # Remove HTML tags (both open and closed)
    data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

    # Remove URLs
    data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

    return data


In [74]:
# TODO: Remove Test Block
# Test Non-alpha characters replace RegEX
import re

# Remove non-alphabetical characters
re.sub(r"\\|[^a-zA-z\s]", " ", "33245 sure# if this % was #not! 55888392923 What do# you will** of it.? $$^!")
re.sub(
    r"\\|[^a-zA-z\s]",
    " ",
    "receive necklace gift disappointed imperfection \\pearls\\ look close enoughthe coating arrive black box inside drawstring bag inside plastic ziploc bag \\made china\\ tagsending back",
)


'      sure  if this   was  not              What do  you will   of it     ^ '

'receive necklace gift disappointed imperfection  pearls  look close enoughthe coating arrive black box inside drawstring bag inside plastic ziploc bag  made china  tagsending back'

In [75]:
# Remove non-alphabetical characters
def remove_non_alpa_characters(data: pd.Series):
    return data.str.replace(r"_+|\\|[^a-zA-z\s]", " ", regex=True)


In [76]:
# TODO: Remove Test Block
# Test remove extra space RegEX
import re

re.sub(r"^\s*|\s\s*", " ", "sameple of the    template, where nothing \t is defined.")


' sameple of the template, where nothing is defined.'

In [77]:
# Remove extra spaces
def remove_extra_spaces(data: pd.Series):
    return data.str.replace(r"^\s*|\s\s*", " ", regex=True)


In [78]:
# Install contractions package, if you don't have it
! pip install contractions

In [79]:
# TODO: Remove Test Block
# Test expanding of contractions
import contractions

sample = "I was disappointed with this purchase, they were extremely small, can't even wear them, they should specify the size, I would've never purchased them if I had known... Thanks"
expanded = " ".join([contractions.fix(word) for word in sample.split()])
expanded


'I was disappointed with this purchase, they were extremely small, cannot even wear them, they should specify the size, I would have never purchased them if I had known... Thanks'

In [80]:
# Expanding contractions
def fix_contractions(data: pd.Series):
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    return data.apply(contraction_fixer)


In [81]:
# A dictionary containing the columns and a list of functions to perform on it in order
# TODO: Think about handling negation.
# TODO: Replace numbers with NUM tag.
data_cleaning_pipeline = {
    "review_body": [
        to_lower,
        remove_accented_characters,
        remove_html_and_url,
        fix_contractions,
        remove_non_alpa_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = sampled_data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = data[col]

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[col] = temp_data


Starting: to_lower
Ended: to_lower
Starting: remove_accented_characters
Ended: remove_accented_characters
Starting: remove_html_and_url
Ended: remove_html_and_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpa_characters
Ended: remove_non_alpa_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [82]:
avg_len_after_cleaning = cleaned_data.review_body.str.len().mean()
f"Avg. length of reviews after cleaning :: {avg_len_after_cleaning}"


'Avg. length of reviews after cleaning :: 184.886776'

In [83]:
# TODO: Remove Test Block
# Checkpoint: Cleaned Data
cleaned_data.to_csv("./data/cleaned.tsv", sep="\t", index=False, encoding="UTF-8")


# Pre-processing


## remove the stop words


In [84]:
# TODO: Remove Test Block
cleaned_data = pd.read_csv("./data/cleaned.tsv", sep="\t")


In [85]:
avg_len_before_preprocessing = cleaned_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews before preprocessing :: {avg_len_before_preprocessing}")


Avg. length of the reviews before preprocessing :: 184.886776


In [86]:
def tokenize(data: pd.Series):
    from nltk.tokenize import word_tokenize

    nltk.download("punkt")

    return data.apply(word_tokenize)


In [87]:
# 1. Tokenize the words
tokenized_data = cleaned_data.copy()
tokenized_data["review_body"] = tokenize(cleaned_data.review_body)
tokenized_data.head()


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,star_rating,review_body
0,1,"[much, smaller, than, regular, pandora, read, ..."
1,1,"[too, small, no, satisfied, need, to, b, bigge..."
2,1,"[i, ordered, this, and, it, was, not, rainbow,..."
3,1,"[terrible, broke, in, days]"
4,1,"[this, is, not, a, locket, it, is, completely,..."


In [88]:
import pickle as pkl

with open("./data/tokenized_data.pkl", "wb") as file:
    pkl.dump(tokenized_data, file)


In [89]:
from typing import List, Set


def remove_stopwords(data: pd.Series):
    """Remove stop words using the NLTK stopwords dictionary

    Args:
        string (str): a document

    Returns:
        str: a document with stopwords removed
    """
    from nltk.corpus import stopwords

    nltk.download("stopwords")

    stopwords = set(stopwords.words())

    def remover(word_list: List[str], stopwords: Set[str]):
        return [word for word in word_list if not word in stopwords]

    return data.apply(lambda word_list: remover(word_list, stopwords))


In [90]:
removed_stopwords = tokenized_data.copy()
removed_stopwords["review_body"] = remove_stopwords(tokenized_data["review_body"])
removed_stopwords.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,star_rating,review_body
0,1,"[smaller, regular, pandora, read, fine, print]"
1,1,"[small, satisfied, b, bigger, money, back, dis..."
2,1,"[ordered, rainbow, rainbow, description, disap..."
3,1,"[terrible, broke, days]"
4,1,"[locket, completely, impossible, insert, photo..."


In [91]:
import pickle as pkl

with open("./data/removed_stopwords.pkl", "wb") as file:
    pkl.dump(removed_stopwords, file)


## perform lemmatization


In [92]:
import pickle as pkl

removed_stopwords = None
with open("./data/removed_stopwords.pkl", "rb") as file:
    removed_stopwords = pkl.load(file)


In [93]:
# 1. Perform POS tagging
def perform_nltk_pos_tag(data: pd.Series):
    from nltk import pos_tag

    nltk.download("averaged_perceptron_tagger")

    return data.apply(pos_tag)


In [94]:
pos_tagged_data = removed_stopwords
pos_tagged_data["review_body"] = perform_nltk_pos_tag(pos_tagged_data["review_body"])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [95]:
def perform_wordnet_lemmatization(data: pd.Series):
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer

    nltk.download("omw-1.4")

    def wordnet_pos_tagger(tag: str):
        if tag.startswith("J"):
            return wordnet.ADJ
        elif tag.startswith("V"):
            return wordnet.VERB
        elif tag.startswith("N"):
            return wordnet.NOUN
        elif tag.startswith("R"):
            return wordnet.ADV
        else:
            return None

    lemmatizer = WordNetLemmatizer()
    lemmatized = list()

    for row in data:

        lemmatized_row = list()

        for word, tag in row:
            wordnet_pos_tag = wordnet_pos_tagger(tag)

            if wordnet_pos_tag is None:
                lemmatized_row.append(word)
            else:
                result = lemmatizer.lemmatize(word, wordnet_pos_tag)
                lemmatized_row.append(lemmatizer.lemmatize(word, wordnet_pos_tag))

        lemmatized.append(lemmatized_row)

    return lemmatized


In [96]:
lemmatized_data = removed_stopwords.copy()
lemmatized_data["review_body"] = perform_wordnet_lemmatization(lemmatized_data["review_body"])
lemmatized_data.head()


[nltk_data] Downloading package omw-1.4 to /Users/aditya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,star_rating,review_body
0,1,"[small, regular, pandora, read, fine, print]"
1,1,"[small, satisfy, b, big, money, back, disatisf..."
2,1,"[order, rainbow, rainbow, description, disappo..."
3,1,"[terrible, break, day]"
4,1,"[locket, completely, impossible, insert, photo..."


In [97]:
# TODO: Remove Test Block
# Save lemmatized data
import pickle as pkl

with open("./data/lemamtized_data.pkl", "wb") as file:
    pkl.dump(lemmatized_data, file)


In [98]:
avg_len_after_preprocessing = lemmatized_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews after preprocessing :: {avg_len_after_preprocessing}")


Avg. length of the reviews after preprocessing :: 12.736944


# TF-IDF Feature Extraction


In [99]:
# TODO: Remove Test Block
# Load lemmatized data
import pickle as pkl

lemmatized_data = None
with open("./data/lemamtized_data.pkl", "rb") as file:
    lemmatized_data = pkl.load(file)


In [100]:
sample = lemmatized_data.loc[
    :5,
]
sample["review_body"].apply(lambda words: " ".join(words))


0                small regular pandora read fine print
1    small satisfy b big money back disatisfied big...
2         order rainbow rainbow description disappoint
3                                   terrible break day
4    locket completely impossible insert photo insi...
5    necklace bracelet cheap rich substantial pictu...
Name: review_body, dtype: object

In [101]:
# Concatenate lemmatized sentences back into one sentence
def concatenator(data: pd.Series):
    return data.apply(lambda words: " ".join(words))


In [102]:
final_data = lemmatized_data.copy()
final_data["review_body"] = concatenator(lemmatized_data["review_body"])
final_data.head()


,star_rating,review_body
0,1,small regular pandora read fine print
1,1,small satisfy b big money back disatisfied big...
2,1,order rainbow rainbow description disappoint
3,1,terrible break day
4,1,locket completely impossible insert photo insi...


In [103]:
final_data[final_data["review_body"].str.len() == 0].groupby("star_rating").count()


,review_body
star_rating,
1,163
2,160
3,370
4,327
5,257


In [105]:
final_data["review_body"].isnull().values.any(), final_data["review_body"].isnull().sum()


(False, 0)

In [108]:
# Drop empty strings
final_data["review_body"].replace("", np.nan, inplace=True)
final_data.dropna(subset=["review_body"], inplace=True)
final_data[final_data["review_body"].str.len() == 0].groupby("star_rating").count()
final_data["review_body"].isnull().values.any(), final_data["review_body"].isnull().sum()


,review_body
star_rating,


(False, 0)

In [119]:
final_data["review_body"].isna().sum()


0

In [110]:
final_data = final_data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(20000))


In [111]:
# TODO: Remove Checkpoint ...
final_data.to_csv("./data/data.tsv", sep="\t", index=False)


In [3]:
# TODO: Remove this block
# Retrieve checkpoint
final_data = pd.read_csv("./data/data.tsv", sep="\t")


In [4]:
# Split the data 80-20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_data, test_size=0.2, stratify=final_data["star_rating"])


In [5]:
(train["review_body"].str.len() == 0).sum()


0

In [6]:
final_data.groupby(["star_rating"]).count()


,review_body
star_rating,
1,20000
2,20000
3,20000
4,20000
5,20000


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(train["review_body"])

X_tfidf_train = vectorizer.transform(train["review_body"])
X_tfidf_test = vectorizer.transform(test["review_body"])
y_train = train["star_rating"]
y_test = test["star_rating"]


TfidfVectorizer()

TODO: Use cross validation. KFold, StratifiedKFold


In [11]:
def calc_metrics(y_true, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "avg.": {"precision": np.mean(precision), "recall": np.mean(recall), "f1": np.mean(f1)},
    }


# Perceptron


In [12]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


Perceptron()

{'precision': array([0.45675355, 0.29471983, 0.28701229, 0.32764953, 0.54054826]),
 'recall': array([0.3855 , 0.41025, 0.2685 , 0.31225, 0.47325]),
 'f1': array([0.4181128 , 0.34301839, 0.27744769, 0.31976446, 0.50466542]),
 'avg.': {'precision': 0.3813366921850422,
  'recall': 0.36995,
  'f1': 0.37260175366582926}}

# SVM


In [15]:
from sklearn.svm import SVC

clf = SVC(gamma="auto")
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


SVC(gamma='auto')

{'precision': array([0.58634379, 0.26867394, 0.27938671, 0.36085879, 0.71889401]),
 'recall': array([0.36925, 0.64025, 0.41   , 0.10925, 0.156  ]),
 'f1': array([0.45313698, 0.3785102 , 0.33232016, 0.16772213, 0.25636812]),
 'avg.': {'precision': 0.44283144866503843,
  'recall': 0.33694999999999997,
  'f1': 0.3176115177903184}}

# Logistic Regression


In [13]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


/Users/aditya/.pyenv/versions/3.10.6/envs/csci544-assignment-01/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

{'precision': array([0.55957734, 0.38410413, 0.37733914, 0.43033123, 0.61705038]),
 'recall': array([0.62225, 0.34675, 0.368  , 0.393  , 0.67675]),
 'f1': array([0.58925189, 0.36447247, 0.37261106, 0.41081929, 0.64552283]),
 'avg.': {'precision': 0.473680443786504,
  'recall': 0.48134999999999994,
  'f1': 0.4765355098782134}}

# Naive Bayes


In [14]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


MultinomialNB()

{'precision': array([0.57384276, 0.3728278 , 0.36901271, 0.42162753, 0.61076747]),
 'recall': array([0.58575, 0.354  , 0.3775 , 0.386  , 0.6665 ]),
 'f1': array([0.57973525, 0.36317004, 0.37320811, 0.40302793, 0.63741781]),
 'avg.': {'precision': 0.4696156537891699,
  'recall': 0.47395000000000004,
  'f1': 0.471311827908986}}